# Convolutions for Images

:label:`sec_conv_layer`


Now that we understand how convolutional layers work in theory,
we are ready to see how they work in practice.
Building on our motivation of convolutional neural networks
as efficient architectures for exploring structure in image data,
we stick with images as our running example.


## The Cross-Correlation Operator

Recall that strictly speaking, *convolutional* layers 
are a (slight) misnomer, since the operations they express
are more accurately described as cross correlations.
In a convolutional layer, an input array
and a *correlation kernel* array are combined
to produce an output array through a cross-correlation operation.
Let's ignore channels for now and see how this works 
with two-dimensional data and hidden representations.
In :numref:`fig_correlation`, 
the input is a two-dimensional array
with a height of 3 and width of 3.
We mark the shape of the array as $3 \times 3$ or ($3$, $3$).
The height and width of the kernel are both $2$.
Note that in the deep learning research community,
this object may be referred to as *a convolutional kernel*, 
*a filter*, or simply the layer's *weights*.
The shape of the kernel window
is given by the height and width of the kernel
(here it is $2 \times 2$).

![Two-dimensional cross-correlation operation. The shaded portions are the first output element and the input and kernel array elements used in its computation: $0\times0+1\times1+3\times2+4\times3=19$. ](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/correlation.svg)

:label:`fig_correlation`


In the two-dimensional cross-correlation operation,
we begin with the convolution window positioned
at the top-left corner of the input array
and slide it across the input array,
both from left to right and top to bottom.
When the convolution window slides to a certain position,
the input subarray contained in that window
and the kernel array are multiplied (elementwise)
and the resulting array is summed up
yielding a single scalar value.
This result gives the value of the output array
at the corresponding location.
Here, the output array has a height of 2 and width of 2
and the four elements are derived from
the two-dimensional cross-correlation operation:

$$
0\times0+1\times1+3\times2+4\times3=19,\\
1\times0+2\times1+4\times2+5\times3=25,\\
3\times0+4\times1+6\times2+7\times3=37,\\
4\times0+5\times1+7\times2+8\times3=43.
$$

Note that along each axis, the output 
is slightly *smaller* than the input.
Because the kernel has width and height greater than one,
we can only properly compute the cross-correlation
for locations where the kernel fits wholly within the image,
the output size is given by the input size $H \times W$
minus the size of the convolutional kernel $h \times w$
via $(H-h+1) \times (W-w+1)$.
This is the case since we need enough space
to 'shift' the convolutional kernel across the image
(later we will see how to keep the size unchanged
by padding the image with zeros around its boundary
such that there is enough space to shift the kernel).
Next, we implement this process in the `corr2d` function,
which accepts the input array `X` and kernel array `K`
and returns the output array `Y`.

But first we will import the relevant libraries.

In [1]:
// %mavenRepo snapshots https://oss.sonatype.org/content/repositories/snapshots/

%maven ai.djl:api:0.6.0
%maven org.slf4j:slf4j-api:1.7.26
%maven org.slf4j:slf4j-simple:1.7.26
%maven net.java.dev.jna:jna:5.3.0
%maven ai.djl.mxnet:mxnet-engine:0.6.0
%maven ai.djl.mxnet:mxnet-native-auto:1.7.0-a

In [2]:
import ai.djl.*;
import ai.djl.engine.Engine;
import ai.djl.ndarray.NDArray;
import ai.djl.ndarray.NDList;
import ai.djl.ndarray.NDManager;
import ai.djl.ndarray.index.NDIndex;
import ai.djl.ndarray.types.DataType;
import ai.djl.ndarray.types.Shape;
import ai.djl.nn.Block;
import ai.djl.nn.ParameterList;
import ai.djl.nn.convolutional.Conv2D;
import ai.djl.training.GradientCollector;
import ai.djl.training.ParameterStore;
import ai.djl.training.initializer.NormalInitializer;
import ai.djl.training.loss.Loss;

In [3]:
public NDArray corr2d(NDArray X, NDArray K){
    int h = (int) K.getShape().get(0);
    int w = (int) K.getShape().get(1);

    NDArray Y = manager.zeros(new Shape(X.getShape().get(0) - h + 1, X.getShape().get(1) - w + 1));

    for(int i=0; i < Y.getShape().get(0); i++){
        for(int j=0; j < Y.getShape().get(1); j++){
            Y.set(new NDIndex(i + "," + j), X.get(i + ":" + (i+h) + "," + j + ":" + (j+w)).mul(K).sum());
        }
    }

    return Y;
}

We can construct the input array `X` and the kernel array `K`
from the figure above
to validate the output of the above implementation
of the two-dimensional cross-correlation operation.

In [4]:
NDManager manager = NDManager.newBaseManager();
NDArray X = manager.create(new float[]{0,1,2,3,4,5,6,7,8}, new Shape(3,3));
NDArray K = manager.create(new float[]{0,1,2,3}, new Shape(2,2));
System.out.println(corr2d(X, K));

## Convolutional Layers

A convolutional layer cross-correlates the input and kernels
and adds a scalar bias to produce an output.
The two parameters of the convolutional layer
are the kernel and the scalar bias.
When training models based on convolutional layers,
we typically initialize the kernels randomly,
just as we would with a fully connected layer.

We are now ready to implement a two-dimensional convolutional layer
based on the `corr2d` function defined above.
In the `ConvolutionalLayer` constructor function,
we declare `weight` and `bias` as the two class parameters.
The forward computation function `forward`
calls the `corr2d` function and adds the bias.
As with $h \times w$ cross-correlation
we also refer to convolutional layers
as $h \times w$ convolutions.

In [5]:
public class ConvolutionalLayer{
    
    private NDArray w;
    private NDArray b;
    
    public NDArray getW(){
        return w;
    }
    
    public NDArray getB(){
        return b;
    }
    
    public ConvolutionalLayer(Shape shape){
        NDManager manager = NDManager.newBaseManager();
        w = manager.create(shape);
        b = manager.randomNormal(new Shape(1));
        w.attachGradient();
    }
    
    public NDArray forward(NDArray X){
        return corr2d(X, w).add(b);
    }
    
}

## Object Edge Detection in Images

Let's take a moment to parse a simple application of a convolutional layer:
detecting the edge of an object in an image
by finding the location of the pixel change.
First, we construct an 'image' of $6\times 8$ pixels.
The middle four columns are black (0) and the rest are white (1).

In [6]:
X = manager.ones(new Shape(6,8));
X.set(new NDIndex(":" + "," + 2 + ":" + 6), 0f);
System.out.println(X);

Next, we construct a kernel `K` with a height of $1$ and width of $2$.
When we perform the cross-correlation operation with the input,
if the horizontally adjacent elements are the same,
the output is 0. Otherwise, the output is non-zero.

In [7]:
K = manager.create(new float[]{1, -1}, new Shape(1,2));

We are ready to perform the cross-correlation operation
with arguments `X` (our input) and `K` (our kernel).
As you can see, we detect 1 for the edge from white to black
and -1 for the edge from black to white.
All other outputs take value $0$.

In [8]:
NDArray Y = corr2d(X, K);
Y

ND: (6, 7) cpu() float32
[[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
 [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
 [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
 [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
 [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
 [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
]


We can now apply the kernel to the transposed image.
As expected, it vanishes. The kernel `K` only detects vertical edges.

In [9]:
corr2d(X.transpose(), K);

## Learning a Kernel

Designing an edge detector by finite differences `[1, -1]` is neat
if we know this is precisely what we are looking for.
However, as we look at larger kernels,
and consider successive layers of convolutions,
it might be impossible to specify
precisely what each filter should be doing manually.

Now let us see whether we can learn the kernel that generated `Y` from `X`
by looking at the (input, output) pairs only.
We first construct a convolutional layer
and initialize its kernel as a random array.
Next, in each iteration, we will use the squared error
to compare `Y` to the output of the convolutional layer.
We can then calculate the gradient to update the weight.
For the sake of simplicity, in this convolutional layer,
we will ignore the bias.

This time, we will use the in-built `Block` and `Conv2D` class from DJL. 

In [10]:
X = X.reshape(1,1,6,8);
Y = Y.reshape(1,1,6,7);

Loss l2Loss = Loss.l2Loss();

In [13]:
Block block = Conv2D.builder()
                .setKernel(new Shape(1, 2))
                .optBias(false)
                .setNumFilters(1)
                .build();

block.setInitializer(new NormalInitializer());
block.initialize(manager, DataType.FLOAT32, X.getShape());

ParameterList params = block.getParameters();
NDArray wParam = params.get(0).getValue().getArray();
wParam.attachGradient();

NDArray lossVal = null;
ParameterStore parameterStore = new ParameterStore(manager, false);

NDArray lossVal = null;

for (int i = 0; i < 10; i++) {

    wParam.attachGradient();

    try (GradientCollector gc = Engine.getInstance().newGradientCollector()) {
        NDArray yHat = block.forward(parameterStore, new NDList(X), true).singletonOrThrow();
        NDArray l = l2Loss.evaluate(new NDList(Y), new NDList(yHat));
        lossVal = l;
        gc.backward(l);
    }

    wParam.subi(wParam.getGradient().mul(0.40f));
    
    if((i+1)%2 == 0){
        System.out.println("batch " + (i+1) + " loss: " + lossVal.sum().getFloat());
    }
}

Note that the error has dropped to a small value after 10 iterations. Now we will take a look at the kernel array we learned.

In [14]:
ParameterList params = block.getParameters();
NDArray wParam = params.get(0).getValue().getArray();
wParam

Indeed, the learned kernel array is moving close
to the kernel array `K` we defined earlier.

## Cross-Correlation and Convolution

Recall our observation from the previous section of the correspondence 
between the cross-correlation and convolution operators.
The figure above makes this correspondence apparent.
Simply flip the kernel from the bottom left to the top right.
In this case, the indexing in the sum is reverted,
yet the same result can be obtained.
In keeping with standard terminology with deep learning literature,
we will continue to refer to the cross-correlation operation
as a convolution even though, strictly-speaking, it is slightly different.

## Summary

* The core computation of a two-dimensional convolutional layer is a two-dimensional cross-correlation operation. In its simplest form, this performs a cross-correlation operation on the two-dimensional input data and the kernel, and then adds a bias.
* We can design a kernel to detect edges in images.
* We can learn the kernel's parameters from data.

## Exercises

1. Construct an image `X` with diagonal edges.
    * What happens if you apply the kernel `K` to it?
    * What happens if you transpose `X`?
    * What happens if you transpose `K`?
1. When you try to automatically find the gradient for the `Conv2D` class we created, what kind of error message do you see?
1. How do you represent a cross-correlation operation as a matrix multiplication by changing the input and kernel arrays?
1. Design some kernels manually.
    * What is the form of a kernel for the second derivative?
    * What is the kernel for the Laplace operator?
    * What is the kernel for an integral?
    * What is the minimum size of a kernel to obtain a derivative of degree $d$?

## [Discussions](https://discuss.mxnet.io/t/2349)

![](../img/qr_conv-layer.svg)